In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import sys
import glob
import pandas as pd
import os
import seaborn as sns

from tqdm import tqdm
from statsmodels.distributions.empirical_distribution import ECDF
from collections import defaultdict
import pickle
import re
import json
from pathlib import Path


from open_spiel.python.algorithms.exploitability import nash_conv, best_response
from open_spiel.python.algorithms.ubc_exploitability import game_tree_size

from open_spiel.python.examples.ubc_plotting_utils import *
from open_spiel.python.examples.ubc_sample_game_tree import sample_game_tree, flatten_trees, flatten_tree
from open_spiel.python.examples.ubc_sample_game_tree import NodeType

from auctions.webutils import *

from open_spiel.python.algorithms.random_agent import RandomAgent
import bokeh
from bokeh.layouts import row, column
from bokeh.plotting import figure, show, output_file, save
from bokeh.io import output_notebook
from bokeh.models import HoverTool, ColumnDataSource, ColorBar, LogColorMapper, LinearColorMapper
from bokeh.transform import linear_cmap, log_cmap
from bokeh.palettes import Category10_10, Magma256, Spectral10, Category20_20
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
from collections import defaultdict
from open_spiel.python.examples.ppo_eval import run_eval
from open_spiel.python.examples.ppo_utils import run_ppo

output_notebook()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Loading BokehJS ...

In [9]:
# OPTIONS
EXPERIMENT_NAME = 'jan12_2'
RUN_NAME = 'jan12_regional_vs_national_reveal-dec15ppo_3-102'
### Load checkpoint
checkpoint = get_checkpoint_by_name(EXPERIMENT_NAME, RUN_NAME)
checkpoint.equilibrium_solver_run.config['device'] = 'cpu'
checkpoint.equilibrium_solver_run.config['use_wandb'] = False
env_and_policy = ppo_db_checkpoint_loader(checkpoint)


In [13]:
def prof():
    return run_eval(env_and_policy, num_samples=10_000, report_freq=1000)

def prof_training():
    timesteps = 100_000
    return run_ppo(env_and_policy, 50_000)

In [14]:
import cProfile, pstats
profiler = cProfile.Profile()
profiler.enable()
# eval_output = prof()
eval_output = prof_training()
profiler.disable()

In [17]:
stats = pstats.Stats(profiler).sort_stats('cumtime')
stats.print_stats()


         65434710 function calls (64616505 primitive calls) in 154.747 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        2    0.000    0.000  154.916   77.458 /home/ubuntu/.pyenv/versions/3.8.2/envs/venv/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3215(run_code)
        2    0.000    0.000  154.916   77.458 {built-in method builtins.exec}
        1    0.000    0.000  154.916  154.916 /tmp/ipykernel_27885/2878371160.py:5(<module>)
        1    0.000    0.000  154.916  154.916 /tmp/ipykernel_27885/1631146118.py:4(prof_training)
        1    0.000    0.000  154.916  154.916 /apps/open_spiel/open_spiel/python/examples/ppo_utils.py:348(run_ppo)
        1    1.168    1.168  154.915  154.915 /apps/open_spiel/open_spiel/python/examples/ppo_utils.py:249(training_loop)
    24960    0.151    0.000  110.459    0.004 /apps/open_spiel/open_spiel/python/vector_env.py:20(step)
   299524   12.249    0.000   75.414  

In [18]:
import pprofile
profiler = pprofile.Profile()
with profiler:
    prof_training()


In [19]:
profiler.print_stats()

Total duration: 621.077s
File: /apps/open_spiel/open_spiel/python/games/clock_auction.py
File duration: 286.258s (46.09%)
Line #|      Hits|         Time| Time per hit|      %|Source code
------+----------+-------------+-------------+-------+-----------
     1|         0|            0|            0|  0.00%|from collections import defaultdict
     2|     49916|     0.104117|  2.08585e-06|  0.02%|from dataclasses import dataclass, field
     3|     49916|     0.118459|  2.37316e-06|  0.02%|from distutils.command.build import build
     4|     49916|    0.0955076|  1.91337e-06|  0.02%|from multiprocessing.sharedctypes import Value
     5|     49916|    0.0942366|   1.8879e-06|  0.02%|from tkinter.messagebox import NO
     6|     49916|    0.0896423|  1.79586e-06|  0.01%|import numpy as np
     7|     49916|     0.089618|  1.79538e-06|  0.01%|import pandas as pd
     8|     49916|    0.0925539|  1.85419e-06|  0.01%|from open_spiel.python.examples.ubc_utils import players_not_me, pulp_solve

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [20]:
profiler.dump_stats('ouptut_jan12.prof')